<h1>BLAST Algorithm<h1>

<h4>Packages required by this implementation: </h4><br>
    1. Biopython - allows reading FASTA format files<br>
    2. TQDM - a progress bar implementation<br>
    3. Pandas - data analysis and storage library that provides DataFrames as a storage structure
    4. Pickle - storing and restoring different variables from and back into python  

In [1]:
from Bio import SeqIO
from tqdm import tqdm
import pandas as pd
import pickle

<h4>Functions<h4>
    <h5>getSeqDB(filename, ftype)</h5>
    Parses through all sequences in a given file and returns a Database in list form.

In [2]:
def getSeqDB(filename, ftype="fasta"):
    geneDB = []
    with open(filename, 'r') as allseqs:
        geneDB = list(SeqIO.parse(allseqs, ftype))
    return geneDB

<h5>getKmers(kemr_size:k, filename)</h5> 
Checks if there is a file for k kmer size availible and loads it. Otherwise creates a k size file.

In [3]:
def getKmers(k, filename):
    try:
        return loadKmers(k, filename)
    except:
        print("constructing kmers database")
        kmers = createKmers(k, getSeqDB(filename))
        saveKmers(kmers, str(k), filename)
        return kmers

<h5>createKmers(kmer_size:k, sequence_list:geneDB)</h5>
    Given a list of sequences, generates the kmers for each and saves it into a dictionary.

In [4]:
def createKmers(k, geneDB):
    kmers = {}
    for seqrec in geneDB:
        seq = seqrec.seq
        for i in range(0, len(seq)-k+1):
            ind = str(seq[i:i+k])
            try: 
                kmers[ind] = kmers[ind] + [(seqrec.id, i)]
            except:
                kmers[ind] = [(seqrec.id , i)]
    return kmers

<h5>saveKmers(kmers, kmer_size:k, filename)</h5>
    Given a dictionary of kmers, kmers, saveKmers saves the list in a file using the Pickle library.
<h5>loadKmers(kmer_size:k, filename)</h5>
    Opens and loads the k-th size file.

In [5]:
def saveKmers(kmers, k, filename):
    with open(str(k)+'_kmers_'+filename+'_kb.pickle', 'wb') as kfile:
        pickle.dump(kmers, kfile)
        
def loadKmers(k, filename):
    with open(str(k)+'_kmers_'+filename+'_kb.pickle', 'rb') as kfile:
        return pickle.load(kfile)

<h5>matchSeq(input_sequence:test, kmers, kmer_size:k)</h5>
Creates a dataframe with each Kmer of the input sequence and matches it to all complete Kmer matches in the database

In [6]:
def matchSeq(test, kmers, k):
    row = {'Kmer': '_', 'SeqID':'None', 'Kmer Ind':-1, 'Seq Ind':-1}
    seqDB = []
    
    for i in range(0, len(test)-k+1):
        ind = test[i:i+k]
        try:
            row['Kmer'] = ind
            row['Kmer Ind'] = i
            for seq in kmers[ind]:
                row['SeqID'] = seq[0]
                row['Seq Ind'] = seq[1]
                seqDB = seqDB+[dict(row)]
        except:
            pass
    return pd.DataFrame(seqDB)

<h5>combineMatches(input_match:seqDB)</h5>
Takes the matched DataFrame and yields the continous sequences from each queried sequence and database sequence in a subsequence object<br><br>

<p>Splits the matched DataFrame by the Sequence IDs<br>
    Reads every row within each splitted DataFrame<br>
    Checks for continuity by comparing each Index with previous index and expecting a difference of 1 for continuity, 0 for different match, and any other number for no more continuity.
    </p>

In [7]:
def combineMatches(seqDB):
    subseq = []
    seqDB = seqDB.sort_values(by='Kmer Ind')

    for s in seqDB["SeqID"].unique():
        tempDB = seqDB[seqDB["SeqID"]==s]
        s_seq = e_seq = []
        temp = []

        for j in range(0, len(tempDB)): 
            row = tempDB.iloc[j]
            rem = []
            toAdd = True

            for i in range(0, len(e_seq)):
                if row['Kmer Ind'] - e_seq[i][0] > 1:
                    rem = rem + [i]
                elif row['Kmer Ind'] == e_seq[i][0]:
                    if row['Seq Ind'] == e_seq[i][1]:
                        toAdd = False
                elif row['Kmer Ind'] - e_seq[i][0] == 1:
                    if row['Seq Ind'] - e_seq[i][1] == 1:
                        e_seq[i] = (row['Kmer Ind'], row['Seq Ind'])
                        toAdd = False
                    elif row['Seq Ind'] == e_seq[i][1]:
                        toAdd = False
                else:
                    print("HELP: ERROR")
            if toAdd:
                e_seq = e_seq + [(row['Kmer Ind'], row['Seq Ind'])]
                s_seq = s_seq + [(row['Kmer Ind'], row['Seq Ind'])]

            for i in range(0, len(rem)):
                temp = temp+[((s_seq[rem[i]-i][0], e_seq[rem[i]-i][0]), (s_seq[rem[i]-i][1], e_seq[rem[i]-i][1]))]
                del s_seq[rem[i]-i]
                del e_seq[rem[i]-i]

        for sS, eS in zip(s_seq, e_seq):
            temp = temp + [((sS[0], eS[0]), (sS[1], eS[1]))]
        subseq = subseq + [(temp, s)]
    return subseq

<h5>rankMatches(subsequence_object:subseq, kmer_size:k)</h5>
Converts the subsequence object into a DataFrame for better readability and conformity to output structure
<br><h6>NOTE: doesn't really rank the matches, just converts it into a format that can be ranked

In [8]:
def rankMatches(subseq, k):
    row = {'matching #': -1, 'inp start':-1, 'inp end':-1, 'inp length':-1, 'Seq ID':'None', 'seq start':-1, 'seq end':-1, 'seq length':-1}
    matchDF = []

    for v in subseq:
        row['Seq ID'] = v[1]
        for s in v[0]:
            row['matching #'] = s[0][1]-s[0][0]+1
            row['inp start'] = s[0][0]
            row['inp end'] = s[0][1]+k
            row['seq start'] = s[1][0]
            row['seq end'] = s[1][1]+k
            row['inp length'] = row['inp end'] - row['inp start']
            row['seq length'] = row['seq end'] - row['seq start']
            matchDF = matchDF + [dict(row)]

    return pd.DataFrame(matchDF)[['Seq ID', 'matching #', 'inp start', 'inp end', 'inp length', 'seq start', 'seq end', 'seq length']].sort_values(by='matching #', ascending=False).reset_index(drop=True)

In [9]:
def main(filename, testseq, k=11, threshold=2):

    kmers = getKmers(k, filename) 
    seqDB = matchSeq(testseq, kmers, k)
#     seqRank = rankSeq(seqDB, threshold)
    subseq = combineMatches(seqDB)
    seqRank = rankMatches(subseq, k)
    
    return seqRank, seqDB

seqRank, seqDB = main("small.fasta", """MSIIGATRLQNDKSDTYSAGPCYAGGCSAFTPRGTCGKDWDLGEQTCASGFCTSQPLCARIKKTQVCGLRYSSKGKDPLVSAEWDSRGAPYVRCTYDADLIDTQAQVDQFVSMFGESPSLAERYCMRGVKNTAGELVSRVSSDADPAGGWCRKWYSAHRGPDQDAALGSFCIKNPGAADCKCINRASDPVYQKVKTLHAYPDQCWYVPCAADVGELKMGTQRDTPTNCPTQVCQIVFNMLDDGSVTMDDVKNTINCDFSKYVPPPPPPKPTPPTPPTPPTPPTPPTPPTPPTPRPVHNRKVMFFVAGAVLVAILISTVRW""", 5, 13)
seqRank

Seq ID  matching #  inp start  inp end  inp length  \
0     sp|Q6GZX3|002L_FRG3G         316          0      320         320   
1     sp|Q6GZX3|002L_FRG3G          17        269      290          21   
2     sp|Q6GZX3|002L_FRG3G          17        272      293          21   
3     sp|Q6GZX3|002L_FRG3G          14        269      287          18   
4     sp|Q6GZX3|002L_FRG3G          14        275      293          18   
5     sp|Q6GZX3|002L_FRG3G          11        278      293          15   
6     sp|Q6GZX3|002L_FRG3G          11        269      284          15   
7     sp|Q6GZX3|002L_FRG3G           8        269      281          12   
8     sp|Q6GZX3|002L_FRG3G           8        281      293          12   
9      sp|Q91G63|034R_IIV6           6        274      284          10   
10     sp|Q91G63|034R_IIV6           6        271      281          10   
11     sp|Q91G63|034R_IIV6           6        277      287          10   
12     sp|Q91G63|034R_IIV6           6        283      293          10   
13     sp|Q91G63|034R_IIV6           6        280      290          10   
14     sp|Q91G63|034R_IIV6           5        269      278           9   
15    sp|Q6GZX3|002L_FRG3G           5        269      278           9   
16    sp|Q6GZX3|002L_FRG3G           5        284      293           9   
17     sp|Q197E9|011L_IIV3           4        261      269           8   
18   sp|Q8R5A3|AB1IP_MOUSE           3        261      268           7   
19     sp|Q91G63|034R_IIV6           3        286      293           7   
20    sp|Q8IZP0|ABI1_HUMAN           2        262      268           6   
21    sp|Q8IZP0|ABI1_HUMAN           2        262      268           6   
22    sp|Q8RZ35|ABI5_ORYSJ           2        261      267           6   
23     sp|O55741|132L_IIV6           2        262      268           6   
24    sp|Q8IZP0|ABI1_HUMAN           2        262      268           6   
25    sp|Q8IZP0|ABI1_HUMAN           2        262      268           6   
26    sp|Q8IZP0|ABI1_HUMAN           2        262      268           6   
27    sp|Q39371|3AP1_BRAOL           2        262      268           6   
28    sp|Q96356|2AP1_BRAOT           2        262      268           6   
29    sp|Q9NYB9|ABI2_HUMAN           2        262      268           6   
..                     ...         ...        ...      ...         ...   
567  sp|Q96I13|ABHD8_HUMAN           1        143      148           5   
568  sp|Q4R584|ABHD8_MACFA           1        143      148           5   
569  sp|Q9UGI9|AAKG3_HUMAN           1        143      148           5   
570  sp|Q17QP1|ABHD8_BOVIN           1        143      148           5   
571   sp|B9FK36|ACC2_ORYSJ           1        152      157           5   
572   sp|B7MMS8|ABDH_ECO45           1        133      138           5   
573   sp|Q8RG99|ACCA_FUSNN           1        156      161           5   
574  sp|Q0SJD4|ACDH1_RHOJR           1        160      165           5   
575   sp|Q9ZI57|ACDH_PSEST           1        160      165           5   
576  sp|Q22347|ACADM_CAEEL           1        162      167           5   
577  sp|Q80YU0|ABHGB_MOUSE           1        163      168           5   
578    sp|Q5XIL6|ABHGB_RAT           1        163      168           5   
579   sp|Q9LF79|ACA8_ARATH           1        249      254           5   
580   sp|Q9LF79|ACA8_ARATH           1        140      145           5   
581   sp|Q0I8N1|ACCA_SYNS3           1        140      145           5   
582  sp|Q14BT6|A4GCT_MOUSE           1        139      144           5   
583  sp|P28286|5HT2B_DROME           1        304      309           5   
584   sp|Q54XC1|5NTC_DICDI           1        138      143           5   
585  sp|Q0RXC4|ACDH6_RHOJR           1        136      141           5   
586  sp|Q0IXP9|AARE1_ORYSJ           1        136      141           5   
587  sp|D4ATR3|A7629_ARTBC           1        135      140           5   
588   sp|P07046|3SHD_NEUCR           1        135      140           5   
589  sp|Q9LJX0|AB19B_ARA

In [ ]:
#Extra Function that ranks the sequences based on  either the # of matched Kmers or the frequency (defined as the sum of distance each Kmer Index is away from Sequence Index divided by the count of Kmers)

def rankSeq(seqDB, t, rCount=True):
    seqDB["Distance"] = seqDB.apply(lambda row: abs(row["Kmer Ind"]-row["Seq Ind"]), axis=1)
    seqRank = seqDB[['SeqID', 'Kmer']].groupby(by="SeqID").count().join(seqDB[['SeqID', 'Distance']].groupby(by="SeqID").sum())
    seqRank["Freq"] = seqRank.apply(lambda row: row["Distance"]/row["Kmer"], axis=1)
    seqRank = seqRank[seqRank['Kmer'].map(int)>t]
    if (rCount):
        return seqRank.sort_values(by="Kmer", ascending=False), seqDB
    else:
        return seqRank.sort_values(by="Freq").iloc[0:10], seqDB